In [30]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math

In [2]:
# training set과 test set의 모든 이미지 파일에 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제해줍니다.

data_path = './dataset/Images/'
train_path = data_path + 'Train/'
val_path = data_path + 'Validate/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)

    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

./dataset/Images/Train/Banh gio/110.jpg
./dataset/Images/Train/Banh gio/468.jpg
./dataset/Images/Train/Banh gio/384.jpg
./dataset/Images/Train/Canh chua/101.jpg
./dataset/Images/Train/Canh chua/499.jpg
./dataset/Images/Train/Canh chua/687.jpg
./dataset/Images/Train/Canh chua/80.jpg
./dataset/Images/Train/Canh chua/92.jpg
./dataset/Images/Train/Canh chua/147.jpg
./dataset/Images/Train/Banh cuon/88.jpg
./dataset/Images/Train/Banh cuon/26.jpg
./dataset/Images/Train/Banh cuon/781.jpg
./dataset/Images/Train/Banh cuon/350.jpg
./dataset/Images/Train/Nem chua/98.jpg
./dataset/Images/Train/Nem chua/314.jpg
./dataset/Images/Train/Nem chua/112.jpg
./dataset/Images/Train/Nem chua/267.jpg
./dataset/Images/Train/Nem chua/281.jpg
./dataset/Images/Train/Nem chua/291.jpg
./dataset/Images/Train/Nem chua/424.jpg
./dataset/Images/Train/Nem chua/422.jpg
./dataset/Images/Train/Nem chua/423.jpg
./dataset/Images/Train/Banh chung/11.jpg
./dataset/Images/Train/Banh chung/5.jpg
./dataset/Images/Train/Banh chung/

In [9]:
#10개만 사용
cls = [x for x in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, x))][:10]
cls

['Banh gio',
 'Canh chua',
 'Banh cuon',
 'Nem chua',
 'Banh chung',
 'Chao long',
 'Banh beo',
 'Banh trang nuong',
 'Banh canh',
 'Banh khot']

In [12]:
train_length = 0
test_length = 0
val_length = 0

for food in cls:
    train_food_path = os.path.join(train_path, food)
    test_food_path = os.path.join(test_path, food)
    val_food_path = os.path.join(val_path, food)
    
    images_train = os.listdir(train_food_path)
    images_test = os.listdir(test_food_path)
    images_val = os.listdir(val_food_path)
    
    train_length += len(images_train)
    test_length += len(images_test)
    val_length += len(images_val)

print('training data의 개수: '+str(train_length))
print('validate data의 개수: '+str(val_length))
print('test data의 개수: '+str(test_length))

training data의 개수: 5540
validate data의 개수: 798
test data의 개수: 1591


In [28]:
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로부터 class label을 만들고, 이미지를 읽는 함수
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def load_data(data_path, cls, img_shape, batch_size=32, is_train=True):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    '''
    #class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    class_names = cls
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)
    DATASET_SIZE = tf.data.experimental.cardinality(list_ds).numpy()

    return ds, DATASET_SIZE
def show_batch(image_batch, label_batch, class_names):
    size = len(image_batch)
    sub_size = int(size ** 0.5) + 1

    plt.figure(figsize=(10, 10), dpi=80)
    for n in range(size):
        plt.subplot(sub_size, sub_size, n+1)
        plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.5)
        plt.title(class_names[np.array(label_batch[n])==True][0].title())
        plt.imshow(image_batch[n])
    plt.show()


In [26]:
batch_size = 16
train_ds, TRAIN_SIZE = load_data(data_path=train_path, cls= cls, img_shape=(224, 224), batch_size=batch_size)
val_ds, VAL_SIZE = load_data(data_path=val_path, cls=cls,img_shape=(224, 224), batch_size=batch_size)

In [31]:
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
steps_per_epoch = compute_steps_per_epoch(TRAIN_SIZE)
val_steps = compute_steps_per_epoch(VAL_SIZE)

# 모델 : EfficientNet

In [50]:
from tensorflow.keras.applications import EfficientNetB0

class myModel(tf.keras.Model):
    '''
    EfficientNetB0을 백본으로 사용하는 모델을 구성합니다.
    Classification 문제로 접근할 것이기 때문에 맨 마지막 Dense 레이어에 
    우리가 원하는 클래스 갯수 만큼을 지정해주어야 합니다.
    '''
    def __init__(self, num_classes=5, freeze=False):
        super(myModel, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        if freeze:
            self.base_model.trainable = False
        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                       tf.keras.layers.BatchNormalization(),
                                       tf.keras.layers.Dropout(0.5, name="top_dropout")])
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")
    def call(self, inputs, training=True):
        x = self.base_model(inputs)
        x = self.top(x)
        x = self.classifier(x)
        return x

if __name__ == '__main__':
    model = myModel(num_classes=10, freeze=True)
    model.build(input_shape=(None, 224, 224, 3))
    print(model.summary())

Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 sequential_6 (Sequential)   (None, 1280)              5120      
                                                                 
 pred (Dense)                multiple                  12810     
                                                                 
Total params: 4067501 (15.52 MB)
Trainable params: 15370 (60.04 KB)
Non-trainable params: 4052131 (15.46 MB)
_________________________________________________________________
None


In [51]:
from tensorflow.keras.utils import Progbar

class Trainer:
    def __init__(self, model, epochs, batch, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    def compute_acc(self, y_pred, y):
        correct = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        return accuracy

    @tf.function
    def train_on_batch(self, x_batch_train, y_batch_train):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)    # 모델이 예측한 결과
            train_loss = self.loss_fn(y_batch_train, logits)     # 모델이 예측한 결과와 GT를 이용한 loss 계산

        grads = tape.gradient(train_loss, model.trainable_weights)  # gradient 계산
        self.optimizer.apply_gradients(zip(grads, model.trainable_weights))  # Otimizer에게 처리된 그라데이션 적용을 요청

        return train_loss, logits

    def train(self, train_dataset, acc_metric, steps_per_epoch, val_dataset, val_step):
        metrics_names = ['train_loss', 'train_acc', 'val_loss']

        for epoch in range(self.epochs):
            print("\nEpoch {}/{}".format(epoch+1, self.epochs))

            train_dataset = train_dataset.shuffle(100)
            val_dataset = val_dataset.shuffle(100)

            train_dataset = train_dataset.take(steps_per_epoch)
            val_dataset = val_dataset.take(val_step)

            progBar = Progbar(steps_per_epoch * self.batch, stateful_metrics=metrics_names)

            train_loss, val_loss = 100, 100

            # 데이터 집합의 배치에 대해 반복합니다
            for step_train, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                train_loss, logits = self.train_on_batch(x_batch_train, y_batch_train)

                # train metric(mean, auc, accuracy 등) 업데이트
                acc_metric.update_state(y_batch_train, logits)

                train_acc = self.compute_acc(logits, y_batch_train)
                values = [('train_loss', train_loss), ('train_acc', train_acc)]
                # print('{}'.format((step_train + 1) * self.batch))
                progBar.update((step_train + 1) * self.batch, values=values)

            for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
                logits = model(x_batch_val, training=False)
                val_loss = self.loss_fn(y_batch_val, logits)
                val_acc = self.compute_acc(logits, y_batch_val)
                values = [('train_loss', train_loss), ('train_acc', train_acc), ('val_loss', val_loss), ('val_acc', val_acc)]
            progBar.update((step_train + 1) * self.batch, values=values, finalize=True)

In [52]:
loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3)
acc_metric = tf.keras.metrics.CategoricalAccuracy()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

model = myModel(num_classes=10)
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(checkpoint, directory=".", max_to_keep=5)

trainer = Trainer(model=model,
                  epochs=3,
                  batch=1,
                  loss_fn=loss_function,
                  optimizer=optimizer,)

trainer.train(train_dataset=train_ds,
            steps_per_epoch=steps_per_epoch,
            val_step=val_steps,
            val_dataset=val_ds,
            acc_metric=acc_metric)


Epoch 1/3


2024-04-08 14:22:51.293415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 337/1094 [========>.....................] - ETA: 2:58 - train_loss: 1.1022 - train_acc: 0.1875 

Corrupt JPEG data: 9 extraneous bytes before marker 0xe2


 759/1094 [===================>..........] - ETA: 1:18 - train_loss: 0.9016 - train_acc: 0.0625

Corrupt JPEG data: 229 extraneous bytes before marker 0xd9


1094/1094 [==============================] - 272s 232ms/step - train_loss: 1.0070 - train_acc: 0.1250


2024-04-08 14:27:13.957608: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-04-08 14:27:14.075667: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-04-08 14:27:16.169183: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-04-08 14:27:16.169198: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: cHRM chunk does not match sRGB


1094/1094 [==============================] - 311s 267ms/step - train_loss: 1.0070 - train_acc: 0.1250 - val_loss: 1.0860 - val_acc: 0.0000e+00

Epoch 2/3
1094/1094 [==============================] - 285s 256ms/step - train_loss: 1.0654 - train_acc: 0.1875 - val_loss: 1.2182 - val_acc: 0.0625

Epoch 3/3
1094/1094 [==============================] - 298s 264ms/step - train_loss: 1.0000 - train_acc: 0.2500 - val_loss: 1.0587 - val_acc: 0.0625


In [60]:
num_classes = 1 
epoch = 1 
batch_size = 16
img_size = 224

checkpoint_path = './checkpoints/'

model = myModel(num_classes=num_classes)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

test, TEST_SIZE = load_data(data_path=test_path, cls=cls, img_shape=(img_size, img_size), batch_size=batch_size, is_train=False)

checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

for step_train, (x_batch_train, y_batch_train) in enumerate(test.take(10)):
    #print(model(x_batch_train))
    prediction = model(x_batch_train)
    #print(tf.argmax(y_batch_train, axis=1))
    #print(tf.argmax(prediction, axis=1))
    #print(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1)))
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))
    #print("Prediction: {}".format(tf.argmax(prediction, axis=1)))

14/16
13/16
11/16
8/16
14/16
10/16
12/16
10/16
14/16


2024-04-08 14:40:40.327556: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


12/16
